In [2]:
from qdrant_client import QdrantClient, models
from fastembed import TextEmbedding
from langchain_qdrant import QdrantVectorStore, FastEmbedSparse, RetrievalMode
from qdrant_client.http.models import Distance, VectorParams
from langchain_community.embeddings.fastembed import FastEmbedEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter, HTMLSectionSplitter
from langchain.storage import InMemoryStore, LocalFileStore
from langchain.retrievers import ParentDocumentRetriever
from langchain.storage._lc_store import create_kv_docstore
from astro_chart import full_chart_generation, get_astro_data


/home/mameuio/zhiweidoushu/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
#Seach Chinese embedding model  
# import json
# for model in TextEmbedding.list_supported_models():
#     if "Chinese" in model["description"]:
#         print(json.dumps(model, indent=2))


# Embedding model 
model_handle = "jinaai/jina-embeddings-v2-base-zh"
embeddings = FastEmbedEmbeddings(model_name=model_handle)
sparse_embeddings = FastEmbedSparse(model_name="Qdrant/BM25")

Fetching 18 files: 100%|██████████| 18/18 [00:01<00:00, 14.46it/s]


In [ ]:
#Create vector store and doc store
url = "http://localhost:6333"
collection_name = "zhiwei_DAG"

#for 1st time
# docs = []  # put docs here
# client  = QdrantClient('http://localhost:6333')

# vectorstore = QdrantVectorStore.from_documents(
#      docs,
#      embeddings,
#      sparse_embedding=sparse_embeddings,
#      url=url,
#      prefer_grpc=True,
#      collection_name=collection_name,
#      retrieval_mode=RetrievalMode.HYBRID
#  )


client = QdrantClient(url=url, prefer_grpc=True)
vectorstore = QdrantVectorStore(
    embedding=embeddings,
    client=client,
    collection_name=collection_name,
    sparse_embedding=sparse_embeddings,
    retrieval_mode=RetrievalMode.HYBRID
)


fs = LocalFileStore("./store_location")
doc_store = create_kv_docstore(fs)

In [6]:
#Chunking
# Read html file (actually not need but I just pratice)
with open('zhiwei book.html','r',encoding='utf-8') as  f_in:
    text_html = f_in.read()

# Define the Splitters ---
# Parent chunk splitter
headers_to_split_on = [("h1", "Header 1")]
html_splitter = HTMLSectionSplitter(headers_to_split_on)
parent_docs  = html_splitter.split_text(text_html)
#parent_docs = parent_docs[0:10]

# Child chunk splitter
child_splitter = RecursiveCharacterTextSplitter(chunk_size=400)
splits = child_splitter.split_documents(parent_docs )

retriever = ParentDocumentRetriever(
    vectorstore=vectorstore,
    docstore=doc_store,
    child_splitter=child_splitter,
    search_kwargs={"k": 10}
)

In [9]:
#Add Document
#retriever.add_documents(parent_docs)

In [10]:
query = "兄弟"
found_docs = retriever.invoke(query)

In [11]:
found_docs

[Document(metadata={'Header 1': '☆19、左辅星、右弼星'}, page_content='☆19、左辅星、右弼星 \n 辅弼二星不是正曜，只是助星，基本性质为先天助力，平辈贵人，助力来自平辈，如同事、同学、朋友、兄弟、合作伙伴，亦可为晚辈。助力是天然的，不必刻意花力气去寻找。左辅较成熟稳重，偏向跑腿；右弼较温柔机谋，偏向顾问幕宾，带不爱约束和桃花色彩，左辅则无。左辅力量较右弼强，故命坐左辅会右弼比命坐右弼会左辅力强。 \n 左右二星无庙陷之分，落诸宫均能降福，四墓宫最佳。二星有多次之意，好坏均主发生多次。命宫有左或右，喜开车。 \n 左右亦宜双星同加会于三方四正宫才有力，只见单星则力弱或主代理工作。左或右单守命宫，命宫无正曜，无自主性，易受人左右，重感情，多主少年离家，或主离宗庶出、过房、随母改嫁、或入赘妻家，若对宫为同巨、阳梁、机阴、机巨、同阴的组合更是，若三方紫府昌曲魁钺合照，仍吉。六亲宫只加会单星，则易有双重的六亲，例如入父母宫主双重父母，入夫妻有二次婚姻，再见煞或加桃花星始验。 \n 左右均主乐观仁厚，即使落宫主星刚克或消极，亦能有所缓解而减轻。 \n 左右若与紫府同宫或拱照或相夹，是良臣逢明主，可达很高 \n 的成就。二星均不喜梁同机巨武等正星。逢天梁不羁，天同爱享受，天机善变，巨门是非，武曲决绝，与左右的本性不相洽，虽能减缓其弱点，但助力因而减弱，或增加感情困扰和波折。二星亦忌与杀破狼同宫，易有感情困扰，婚姻波折。读书限内遇左右，容易补考重考辍学等。逢化科，善思考，善策划，有声誉。 \n 辅弼分居于命迁宫者，终身福厚；分守命身宫亦吉，在命宫或三方照会稍次。分守命夫宫，无煞亦吉，有煞婚姻不顺。 \n 左右夹命为贵格，如安命丑宫，九月十一月生人；安命未宫，三月五月生人，得左右夹命，不富则贵，不是这两月生人亦吉。亦宜主星庙旺无煞，不然不贵，仅主有人缘而已。紫破、天府、阴阳等组合，最喜左右夹，能增加地位和人生的稳定性。煞忌逢左右夹，亦可变不利为有利。若同时得龙池凤阁夹力量更强。 \n 左右同宫守命，披罗衣紫，辰戌宫安命，逢正月七月生人；丑宫安命，九月生人；已宫安命，四月生人；卯亥宫安命，六月十二月生人，即是。不是这几个月生人亦吉。主得助力，事业顺遂。为人忠厚老诚，逢吉终生福厚寿长。 \n 命坐辰戌，得左右对照助力亦很大，可冲破罗

In [12]:
from dotenv import load_dotenv
load_dotenv()

True

In [13]:
# Import relevant functionality
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage, SystemMessage
from langchain_core.prompts import ChatPromptTemplate


llm = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0.5
)



In [14]:
from langchain.prompts import PromptTemplate

system_prompt_text = """
角色: 你是一个专业的紫微斗数检索查询生成器，用于 RAG(检索增强生成)。

目标: 基于用户的命盘关键信息与具体问题，生成若干高质量、可直接用于检索的查询字符串，覆盖问题最核心的宫位与关键星曜组合，并按相关性排序。

约束:
- 仅使用用户的命盘中出现的宫位、主星、辅星、煞星；不要编造。
- 若用户的命盘未提供四化，不要构造四化查询；若提供，至少包含一条带四化的查询。
- 使用标准中文术语与命名；短语之间用空格分隔；避免冗余词。
- 每条查询尽量精炼但信息完整；生成 1-3 条，按相关性从高到低排序。
- 仅输出 JSON 字符串数组；不要任何解释、标题或 Markdown 代码块。

步骤:
1) 解析问题与领域（如事业财运、感情婚姻、健康、人际等），确定最核心宫位。
2) 从命盘中提取该核心宫位及其三方四正的关键主星、**如果宮位沒有核心（空宮）主星，用對面宮位的主星來代替**, 重要辅星（左辅、右弼、文昌、文曲）、煞星（擎羊、陀罗、火星、铃星、地空、地劫。
3) 组合查询，覆盖核心宫位、关键星曜影响，去重并按相关性排序。

查询模板（仅使用以下几类，其一或多条）:
- 模式A: 宫位-星曜-主题
  形式: [宫位] [星曜组合] [问题领域] 解释
  例: 夫妻宫 廉贞七杀 感情婚姻 解释
- 模式B: 星曜-四化-宫位
  形式: [星曜] 化[禄/权/科/忌]入 [宫位] 对[问题领域]的影响
  例: 太阳化忌入父母宫 对学业的影响
- 模式C: 宫位-煞/辅
  形式: [宫位] 遇 [煞星/辅星] 作用
  例: 命宫 擎羊同宫 影响
- 模式D: 宫位关系/格局
  形式: [宫位A] [宫位B] [关系类型] 影响
      或 [星曜组合] [宫位] [格局名称] 格局
  例: 命宫 迁移宫 对照 影响
      紫微破军在丑未宫 紫府朝垣格

输入:
- 用户命盘: {user_stars}
- 用户问题: {user_question}

输出（严格遵守）:
- 仅输出 JSON 字符串数组；不含任何解释、前后缀、注释或代码块标记。
- 项目去重、语义多样化、按相关性降序。

示例输出:
[\"夫妻宫 廉贞七杀 感情婚姻 解释\",
 \"廉贞化忌入夫妻宫 对婚姻的影响\",
 \"夫妻宫三方四正 福德宫 天府 影响\",
 \"命宫 紫微 感情观\",
 \"七杀星 感情 特质\"]
"""

ziwei_prompt_template_classic = PromptTemplate(
    input_variables=["user_question","user_stars"],
    template=system_prompt_text
)


chain = ziwei_prompt_template_classic | llm


In [15]:
question = "我什么时候能遇到正缘？"
stars = full_chart_generation("1994-8-25", 11, "男")

ai_message = chain.invoke({"user_question": question,"user_stars":  stars})

In [16]:
print(ai_message.content)

["夫妻宫 天钺 天姚 感情婚姻 解释","命宫 廉贞破军 感情婚姻 解释","夫妻宫 遇 天钺 作用"]


In [17]:
import json

def query_extraction(raw_output_string):
    start_index = raw_output_string.find('[')
    end_index = raw_output_string.rfind(']')

    # 如果找到了 '[' 和 ']'，就提取它们之间的内容
    if start_index != -1 and end_index != -1:
        clean_json_string = raw_output_string[start_index : end_index + 1]
        try:
            # 2. 使用 json.loads() 将清理后的字符串解析为 Python 列表
            queries_list = json.loads(clean_json_string)
            return queries_list
        except json.JSONDecodeError as e:
            print(f"解析失败: {e}")

    else:
        print("在字符串中没有找到有效的 JSON 数组 ('[...]')。")
        return []


In [20]:
query = query_extraction(ai_message.content)[0]

In [21]:
from langchain.retrievers import ContextualCompressionRetriever
from langchain_community.document_compressors import FlashrankRerank
from langchain_openai import ChatOpenAI

In [22]:
from langchain.retrievers.contextual_compression import ContextualCompressionRetriever
from langchain_community.document_compressors.rankllm_rerank import RankLLMRerank

compressor = FlashrankRerank()
compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor, base_retriever=retriever
)
test123 = compression_retriever.invoke(query)

INFO:flashrank.Ranker:Downloading ms-marco-MultiBERT-L-12...
ms-marco-MultiBERT-L-12.zip: 100%|██████████| 98.7M/98.7M [00:02<00:00, 43.7MiB/s]


In [27]:
def pretty_print_docs(docs, save_path=None):
    doc_text = f"\n{'-' * 100}\n".join(
        [f"Document {i + 1}:\n{d.page_content}" for i, d in enumerate(docs)]
    )
    print(doc_text)
    if save_path:
        with open(save_path, 'w', encoding='utf-8') as f_out:
            f_out.write(doc_text)  # allow saving retrieved context
    return doc_text


In [ ]:
answer_prompt_text = """
角色: 你是专业紫微斗数顾问，任务是基于检索到的文段整理关键信息，供后续回答使用。

目标: 在不直接回答用户问题的前提下，先归纳文段中与命盘和提问相关的要点。

输入:
- 用户命盘: {user_chart}（结构化的宫位、主星、辅星、煞星、四化等）
- 用户问题: {user_question}
- 检索文段: {retrieved_passages}（列表; 每段包含 id, title, content, source）

指令:
1. 阅读全部文段，挑出与用户问题及命盘相关的关键信息；忽略无关材料。
2. 将信息按主题分组，并以精炼语句概述；每个要点注明引用来源 [#id]。
3. 指出这些要点与命盘要素（宫位、主星、三方四正、四化等）的关联，或说明缺少对应信息。
4. 如存在重要空缺或需要额外检索的线索，单独列出。
5. 不做结论或建议；不要编造命盘或文段外的信息。

输出格式:
主题脉络:
- … [#id]
- …
命盘关联:
- …
- …
信息缺口:
- …
"""


In [28]:
pretty_print_docs(test123)

Document 1:
★二、看桃花 
 桃花，是指有异性情缘、外遇。有桃花不一定会婚姻不顺，要看具体的格局而定，一般，桃花轻者仅主浪漫一点，桃花过重则变为淫乱。 
 桃花星有：正星的贪狼、廉贞为甲级桃花星，守命宫不加桃花星已是多情种子，再加一二次级桃花星就是重桃花了，此时才能充份发挥其在感情的桃花作用；紫微、破军守命宫的桃花稍次于贪廉。天姚、咸池、沐浴、红鸾、天喜、生年化禄星和化科星、六吉星均为次级桃花星。凡主星加上桃花星就易有桃花现象，桃花星加得越 
 多桃花性就越重，但须有煞忌星才会显得迷乱或是滥情，无煞忌星则较不会乱来。 
 天梁、七杀、武曲守命的人，最缺乏桃花，其次是巨门、天府守命的人。 
 生年命盘有桃花格局的人，一生较容易有桃花；生年命盘无桃花格局的人，行限逢桃花，青年未婚者可有姻缘，已婚者并非有桃花，而只是夫妻感情好些罢了。 
 命宫有粒把桃花星，其他多为正气之星且无煞，夫妻宫无问题，多主夫妻感情好，尤其女命，会有风度情趣，生活和谐。若夫妻宫又有桃花星飞入时，则会有桃花纠纷事，婚姻将不美满。 
 行运逢桃花格局或桃花的组合，不必看夫妻宫就可直断其有恋爱感情事发生。 
 桃花星聚于不同的宫度则有不同的象意：命宫的桃花既有思想又有行动，子女宫的桃花侧重于色欲方面，夫妻宫的桃花侧重于恋爱和外遇同居方面，疾厄宫的桃花侧重于身体接触方面而多主肉欲型桃花，福德宫的桃花侧重于思想而少行动，奴仆宫的桃花侧重于外来诱惑方面，身宫的桃花是外貌美及吸引力。 
 桃花有内桃花和外桃花之分。内桃花主夫妻间较浪漫和性生活较频繁，如昌曲星之类。外桃花则是主不正常的婚外恋情，如天姚、咸池之类。 
 逢桃花而致破财、出丑、婚姻不利者，称为“桃花劫”。桃花星逢空劫大耗，或财禄星逢冲，就易发生“桃花劫”。但贪狼、廉贞见空劫有减弱桃花而转化为工作艺术的作用，其它桃花星则无变化。 
 逢桃花而致官司判狱，称为“桃花煞”。桃花星逢羊陀刑白虎官符破碎及其他刑狱的组合等，就容易发生“桃花煞”。 
 夫妻宫有沐浴咸池，主有实质性的桃花（肉欲桃花）；廉贞已亥（男女同论）会煞星，也是实质性的肉欲桃花；贪居亥子遇羊陀为泛水桃花，亦为肉欲桃花；廉贞会咸池又会文曲，是精神性桃花(即友谊性而非色情)；贪狼会陀罗为风流彩杖，是肉欲桃花，主有酒色事件；同巨会文曲，为精神性桃花，女命可能有实质性桃花；廉巨会天姚，易有